In [1]:
import os    as os

import numpy as np

import xarray as xr

import pandas as pd


In [2]:
def days_per_year(year):
    if (year % 4) == 0:
        if (year % 100) == 0:
            if (year %  400) == 0:
                days_per_year = 366
            else:
                days_per_year = 365
        else:
            days_per_year = 366
    else:
        days_per_year = 365
    return days_per_year

In [3]:
working_directory  = "./"

loca2_inv_file     = "./LOCA2_Model_Member_Available_List.csv"

globus_root_dir    = "/data/GLOBUS/wjc"

input_root_dir   = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/"
output_root_dir  = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Climate_CONUS/Monthly/"
variables          = ["tasmax", 
                      "tasmin", 
                      "pr"]

scenarios          = ["historical", 
                      "ssp245", 
                      "ssp370", 
                      "ssp585"]


In [4]:
loca2_ensembles_list = pd.read_csv(filepath_or_buffer = loca2_inv_file)
models  = loca2_ensembles_list[ "Model"].values
members = loca2_ensembles_list["Member"].values

display(loca2_ensembles_list)



,Model,Member,Var,Season,Stat,NetError,Rank,historical,ssp245,ssp370,ssp585,n_complete_ens,n_avail_ens
0,GFDL-ESM4,r1i1p1f1,All,Annual,Both,6.152235,1,XNP,XNP,XNP,XNP,4,4
1,BCC-CSM2-MR,r1i1p1f1,All,Annual,Both,6.357673,1,XNP,XNP,XNP,XNP,4,4
2,GFDL-CM4,r1i1p1f1,All,Annual,Both,6.498461,1,XNP,XNP,---,XNP,3,3
3,CNRM-CM6-1,r1i1p1f2,All,Annual,Both,6.513064,1,XNP,XNP,XNP,XNP,4,4
4,TaiESM1,r1i1p1f1,All,Annual,Both,6.658829,1,XNP,XNP,XNP,---,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,EC-Earth3-Veg,r1i1p1f1,All,Annual,Both,12.301219,4,XNP,XNP,XNP,XNP,4,4
65,MPI-ESM1-2-HR,r1i1p1f1,All,Annual,Both,12.387494,4,XNP,XNP,XNP,XNP,4,4
66,CESM2-LENS,r1i1p1f1,All,Annual,Both,11.249889,6,XNP,---,XNP,---,2,2
67,IPSL-CM6A-LR,r1i1p1f1,All,Annual,Both,11.812705,6,XNP,XNP,XNP,XNP,4,4


In [5]:
meta_script_name = "./CDO_MONAVG.csh"
f0 = open(file     = meta_script_name, 
         mode      = 'w', 
         encoding  = "utf-8")


for m in range(len(models)):
    
    print("# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$", file = f0)
    
    model  = models[m]
    member = members[m]
    
    meta_deleteme = True
    
    for scenario in scenarios:
        
        print("# ================================================", file = f0)
        
        inventory = loca2_ensembles_list.iloc[m].loc[scenario]
        
        input_file = input_root_dir                              + \
                     scenario + "/"                              + \
                     "LOCA2-CONUS___"                            + \
                     loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                     loca2_ensembles_list.at[m,"Member"] + "___" + \
                     scenario + ".nc"
        
        interim_daily_files  = output_root_dir                             + \
                               scenario + "/"                              + \
                               "LOCA2-CONUS-DAILY___*___"                  + \
                               loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                               loca2_ensembles_list.at[m,"Member"] + "___" + \
                               scenario + ".nc"
        
        cod_climate_files = output_root_dir                             + \
                                scenario + "/"                              + \
                                "LOCA2-CONUS-MONTHLY___*___"                 + \
                                loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                                loca2_ensembles_list.at[m,"Member"] + "___" + \
                                scenario + ".nc"
        
        final_file = output_root_dir                             + \
                     scenario + "/"                              + \
                     "LOCA2-CONUS-MONTHLY___"                     + \
                     loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                     loca2_ensembles_list.at[m,"Member"] + "___" + \
                     scenario + ".nc"
        
        
        if (inventory != "---"):
            
            print("# ",model,member,scenario,inventory, file = f0)
            
            if ("P" in inventory):
                
                    print("# ------------------------------------------------", file = f0)
                    
                    variable    = "pr"
                    
                    interim_daily_file =  output_root_dir           + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-DAILY___" + variable   + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    command_isolate   = "ncks -v " +  variable   + " " + input_file + " " + interim_daily_file
                    print(command_isolate,   file = f0)

                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_SUM___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command       = "nohup cdo --no_history -f nc4 -z zip_8 monsum "
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                    
                    print(command_aggregate, file = f0)  
                    print('nohup ncatted -h -O -a  cell_methods,pr,m,c,"time: sum within days  time: sum over months" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command       = "nohup cdo --no_history -f nc4 -z zip_8 monmax "
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                    
                    print(command_aggregate, file = f0)  
                    print('nohup ncatted -h -O -a  cell_methods,pr,m,c,"time: sum within days  time: maximum over months" ' + cod_climate_file, file=f0)

                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_SUM___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command       = "nohup cdo --no_history -f nc4 -z zip_8 yearsum "
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                    
                    print(command_aggregate, file = f0)  
                    print('nohup ncatted -h -O -a  cell_methods,pr,m,c,"time: sum within days  time: sum over months" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command       = "nohup cdo --no_history -f nc4 -z zip_8 yearmax "
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                    
                    print(command_aggregate, file = f0)
                    print('nohup ncatted -h -O -a  cell_methods,pr,m,c,"time: sum within days  time: maximum over years" ' + cod_climate_file, file=f0)

                    
            if ("X" in inventory):
                
                    print("# ------------------------------------------------", file = f0)
                    
                    variable    = "tasmax"
                                                                          
                    interim_daily_file =  output_root_dir           + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-DAILY___" + variable   + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    command_isolate   = "ncks -v " +  variable   + " " + input_file + " " + interim_daily_file
                    print(command_isolate,   file = f0)
               
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MEAN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmean "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)    
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: mean over months" ' + cod_climate_file, file=f0)

                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmax "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: maximum over months" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MIN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmin "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: minimum over months" ' + cod_climate_file, file=f0)

                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MEAN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmean "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0) 
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: mean over year" ' + cod_climate_file, file=f0)
                    
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmax "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: maximum over year" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MIN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmin "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)    
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,m,c,"time: maximum within days  time: minimum over year" ' + cod_climate_file, file=f0)
                    
            if ("N" in inventory):
                
                    print("# ------------------------------------------------", file = f0)
                    
                    variable    = "tasmin"
                    interim_daily_file =  output_root_dir           + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-DAILY___" + variable   + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    command_isolate   = "ncks -v " +  variable   + " " + input_file + " " + interim_daily_file
                    print(command_isolate,   file = f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MEAN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmean "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)    
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: mean over months" ' + cod_climate_file, file=f0)

                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmax "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: maximum over months" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-MONTHLY_MIN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 monmin "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: minimum over months" ' + cod_climate_file, file=f0)

                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MEAN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmean "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0) 
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: mean over year" ' + cod_climate_file, file=f0)
                    
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MAX___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmax "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)                   
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: maximum over year" ' + cod_climate_file, file=f0)
                
                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         + \
                        scenario + "/"                              + \
                        "LOCA2-CONUS-ANNUAL_MIN___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario + ".nc"
                    cdo_command = "nohup cdo --no_history -f nc4 -z zip_8 yearmin "                    
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                       
                    print(command_aggregate, file = f0)    
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,m,c,"time: maximum within days  time: minimum over year" ' + cod_climate_file, file=f0)

        print("#  ...............................................", file = f0)

        command_clean    = "rm -v  " + interim_daily_files
        print(command_clean,    file = f0)
        
print("# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$", file = f0)

f0.close()
os.system("chmod a+x " + meta_script_name)            

        



0

In [6]:
print("end!")

end!
